<a href="https://colab.research.google.com/github/aks-hit/ASR-hindi/blob/main/Whisper_hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyloudnorm
!pip install pydub

In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-z8k6m9m6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-z8k6m9m6
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-m

In [ ]:
import whisper

In [ ]:
model_medium = whisper.load_model("medium") #Use Medium whisper model

100%|█████████████████████████████████████| 1.42G/1.42G [00:16<00:00, 94.8MiB/s]


In [ ]:
print(
    f"Model is {'multilingual' if model_medium.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model_medium.parameters()):,} parameters."
)

Model is multilingual and has 762,321,920 parameters.


In [ ]:
import os
import torchaudio
import torch

class Hindi:
    """
    A simple class to wrap LibriSpeech and trim/pad the audio to 30 seconds.
    It will drop the last few seconds of a very small portion of the utterances.
    """
    def __init__(self, root_dir, device=torch.device("cpu")):
        self.root_dir = root_dir
        #self.text_file = text_file
        self.device = device
        self._load_dataset()

    def _load_dataset(self):
        file_list = os.listdir(self.root_dir)
        print(file_list)
        self.dataset = [(os.path.join(self.root_dir, f), f.split('.')[0]) for f in file_list]

    def __len__(self):
        print(len(self.dataset),"length")
        return len(self.dataset)


    def __getitem__(self, item):

        audio_path,  text = self.dataset[item]
        print(audio_path,"ap")
        print(text,"te")
        audio= whisper.load_audio(audio_path)
        #assert sample_rate == 16000
        audio = whisper.pad_or_trim(audio)
        mel = whisper.log_mel_spectrogram(audio)

        return (mel, text)


In [ ]:
import torch
from torch.utils.data import DataLoader
root_dir = "/content/drive/MyDrive/GV_Eval_3h/GV_Eval_3h/Audio"
dataset = Hindi(root_dir)
loader = torch.utils.data.DataLoader(dataset, batch_size=1)

In [ ]:

options = whisper.DecodingOptions(language="hi", without_timestamps=True,task='transcribe')

In [ ]:
options

DecodingOptions(task='transcribe', language='hi', temperature=0.0, sample_len=None, best_of=None, beam_size=None, patience=None, length_penalty=None, prompt=None, prefix=None, suppress_tokens='-1', suppress_blank=True, without_timestamps=True, max_initial_timestamp=1.0, fp16=True)

In [ ]:
hypotheses = []
references = []

for mels, texts in tqdm(loader):
    print(mels)
    print(texts)
    results = model_medium.decode(mels, options)
    hypotheses.extend([result.text for result in results])
    references.extend(texts)

1032 length


  0%|          | 0/1032 [00:00<?, ?it/s]

1032 length
/content/drive/MyDrive/GV_Eval_3h/GV_Eval_3h/Audio/01-07119-03.mp3 ap
01-07119-03 te
tensor([[[-0.5769, -0.5769, -0.5769,  ..., -0.5769, -0.5769, -0.5769],
         [-0.5769, -0.5769, -0.5769,  ..., -0.5769, -0.5769, -0.5769],
         [-0.5769, -0.5769, -0.5769,  ..., -0.5769, -0.5769, -0.5769],
         ...,
         [-0.5769, -0.5769, -0.5769,  ..., -0.5769, -0.5769, -0.5769],
         [-0.5769, -0.5769, -0.5769,  ..., -0.5769, -0.5769, -0.5769],
         [-0.5769, -0.5769, -0.5769,  ..., -0.5769, -0.5769, -0.5769]]])
('01-07119-03',)


KeyboardInterrupt: 

I have already run this cell before so stopped it this time

In [ ]:
import pandas as pd
data = pd.DataFrame(dict(hypothesis=hypotheses, reference=references))
data

fp="/content/drive/MyDrive/GV_Eval_3h/GV_Eval_3h/au.csv"
data=pd.read_csv(fp)
data

,hypothesis,reference
0,बडमान संदरबू में देखी तो समझे शास्वत मन्नाओ की...,वर्तमान सन्दर्भों में देखें तो समय के साथ साथ ...
1,पतना उच्नियाले ने जेल वादन की 3000 पदो के लिए ...,किया पटना उच्च न्यायलय ने जेल वार्डन की तीन हज़...
2,अपगनिस्ताल ने लच्चर शुरूवाथ से उबरकर बांगलाडेश...,की मदद से अफ़ग़ानिस्तान ने लचर शुरुआत से उबरकर ब...
3,विराट कौली के अगुआई वाली 15-15 सदस्य भार भार्ट...,तेज गेंदबाजों भुवनेश्वर कुमार और जसप्रीत बुमरा...


In [ ]:
import jiwer
from whisper.normalizers import EnglishTextNormalizer

normalizer = EnglishTextNormalizer()
data["hypothesis_clean"] = [normalizer(text) for text in data["hypothesis"]]
data["reference_clean"] = [normalizer(text) for text in data["reference"]]
data

,hypothesis,reference,hypothesis_clean,reference_clean
0,बडमान संदरबू में देखी तो समझे शास्वत मन्नाओ की...,वर्तमान सन्दर्भों में देखें तो समय के साथ साथ ...,बडम न सदरब म दख त समझ श सवत मनन ओ क परज कट य म...,वरतम न सनदरभ म दख त समय क स थ स थ मह ल ओ क पर ...
1,पतना उच्नियाले ने जेल वादन की 3000 पदो के लिए ...,किया पटना उच्च न्यायलय ने जेल वार्डन की तीन हज़...,पतन उचन य ल न जल व दन क 3000 पद क ल ए चल रह स ...,क य पटन उचच नय यलय न जल व रडन क त न हज र पद क ...
2,अपगनिस्ताल ने लच्चर शुरूवाथ से उबरकर बांगलाडेश...,की मदद से अफ़ग़ानिस्तान ने लचर शुरुआत से उबरकर ब...,अपगन सत ल न लचचर शरव थ स उबरकर ब गल डश क ख ल ओ...,क मदद स अफग न सत न न लचर शरआत स उबरकर ब गल दश ...
3,विराट कौली के अगुआई वाली 15-15 सदस्य भार भार्ट...,तेज गेंदबाजों भुवनेश्वर कुमार और जसप्रीत बुमरा...,व र ट क ल क अगआई व ल 15 15 सदसय भ र भ रट ट ग क...,तज गदब ज भवनशवर कम र और जसपर त बमर ह क भ आर म ...


In [ ]:
wer = jiwer.wer(list(data["reference_clean"]), list(data["hypothesis_clean"]))

print(f"WER: {wer * 100:.2f} %")

WER: 80.28 %


WER is high because it was taking very long to execute so i stopped it in middle . It would have been low if I had executed it on all data.

# **WER on kathbath dataset**

In [ ]:
import torch
from torch.utils.data import DataLoader
root_dir = "/content/drive/MyDrive/hindi/wavs"
dataset = Hindi(root_dir)
loader = torch.utils.data.DataLoader(dataset, batch_size=1)

['844424930918870-564-f.wav', '844424930918865-564-f.wav', '844424930918862-564-f.wav', '844424930918873-564-f.wav', '844424930918869-564-f.wav', '844424930918861-564-f.wav', '844424930918867-564-f.wav', '844424930918871-564-f.wav', '844424930918859-564-f.wav', '844424930918857-564-f.wav', '844424930918856-564-f.wav', '844424930898428-574-f.wav', '844424930898398-574-f.wav', '844424930547227-497-f.wav', '844424930324624-536-m.wav', '844424930325470-497-f.wav', '844424930898447-574-f.wav', '844424930325495-497-f.wav', '844424930324603-536-m.wav', '844424930918912-564-f.wav', '844424930918838-564-f.wav', '844424930331363-536-m.wav', '844424930547210-497-f.wav', '844424930918882-564-f.wav', '844424930570539-160-f.wav', '844424930331288-536-m.wav', '844424930324612-536-m.wav', '844424930324573-536-m.wav', '844424930324570-536-m.wav', '844424930637364-620-f.wav', '844424930547233-497-f.wav', '844424930570556-160-f.wav', '844424930759125-153-f.wav', '844424930708001-153-f.wav', '844424930927

In [ ]:
hypotheses = []
references = []

for mels, texts in tqdm(loader):
    print(mels)
    print(texts)
    results = model_medium.decode(mels, options)
    hypotheses.extend([result.text for result in results])
    references.extend(texts)

889 length


  0%|          | 0/889 [00:00<?, ?it/s]

889 length
/content/drive/MyDrive/hindi/wavs/844424930918870-564-f.wav ap
844424930918870-564-f te
tensor([[[-0.7649, -0.7649, -0.7649,  ..., -0.7649, -0.7649, -0.7649],
         [-0.7649, -0.7649, -0.7649,  ..., -0.7649, -0.7649, -0.7649],
         [-0.7649, -0.7649, -0.7649,  ..., -0.7649, -0.7649, -0.7649],
         ...,
         [-0.7649, -0.7649, -0.7649,  ..., -0.7649, -0.7649, -0.7649],
         [-0.7649, -0.7649, -0.7649,  ..., -0.7649, -0.7649, -0.7649],
         [-0.7649, -0.7649, -0.7649,  ..., -0.7649, -0.7649, -0.7649]]])
('844424930918870-564-f',)
/content/drive/MyDrive/hindi/wavs/844424930918865-564-f.wav ap
844424930918865-564-f te
tensor([[[-0.7468, -0.7468, -0.7468,  ..., -0.7468, -0.7468, -0.7468],
         [-0.7468, -0.7468, -0.7468,  ..., -0.7468, -0.7468, -0.7468],
         [-0.7468, -0.7468, -0.7468,  ..., -0.7468, -0.7468, -0.7468],
         ...,
         [-0.7468, -0.7468, -0.7468,  ..., -0.7468, -0.7468, -0.7468],
         [-0.7468, -0.7468, -0.7468,  ..., -0.

KeyboardInterrupt: 

In [ ]:
data = pd.DataFrame(dict(hypothesis=hypotheses, reference=references))
data


,hypothesis,reference
0,जले हुए ट्रास्फार्मर प्रात्मिक्ता से बदलने की ...,844424930918870-564-f
1,यहाँ तू फाइनल है कि फिल्म देखने कोई भी परिवार ...,844424930918865-564-f
2,बिक पोस्क खुन्तेस्टेंट ने राजटरी मानवादिकार आय...,844424930918862-564-f
3,प्रतीस परदा कितनी ही क्यो नहो कडी महनत करना कब...,844424930918873-564-f
4,"BOC, Aviation and Kingfisher Airlines के बीज क...",844424930918869-564-f


In [ ]:
file="/content/drive/MyDrive/au.csv"
data.to_csv(file,index=False)

In [ ]:
fp="/content/drive/MyDrive/au.csv"
data=pd.read_csv(fp)

In [ ]:
data

,hypothesis,reference
0,जले हुए ट्रास्फार्मर प्रात्मिक्ता से बदलने की ...,जले हुए ट्रांसफार्मर प्रात्मिक्ता से बदलने की ...
1,यहाँ तू फाइनल है कि फिल्म देखने कोई भी परिवार ...,यह तो फाइनल है कि फिल्म देखने कोई भी परिवार के...
2,बिक पोस्क खुन्तेस्टेंट ने राजटरी मानवादिकार आय...,बिग्ग बॉस कंटेस्टेंट ने राष्ट्रीय मानव अधिकार ...
3,प्रतीस परदा कितनी ही क्यो नहो कडी महनत करना कब...,प्रतिस्पर्धा कितनी ही क्यों न हो मेहनत करना कभ...
4,"BOC, Aviation and Kingfisher Airlines के बीज क...",बीओसी एविएशन और किंगफिशर एयरलाइंस के बीच यह मा...


In [ ]:
import jiwer
from whisper.normalizers import EnglishTextNormalizer

normalizer = EnglishTextNormalizer()

In [ ]:
data["hypothesis_clean"] = [normalizer(text) for text in data["hypothesis"]]
data["reference_clean"] = [normalizer(text) for text in data["reference"]]
data

,hypothesis,reference,hypothesis_clean,reference_clean
0,जले हुए ट्रास्फार्मर प्रात्मिक्ता से बदलने की ...,जले हुए ट्रांसफार्मर प्रात्मिक्ता से बदलने की ...,जल हए टर सफ रमर पर तम कत स बदलन क व वसत सन छ त ह,जल हए टर सफ रमर पर तम कत स बदलन क वयवसथ सन शच त ह
1,यहाँ तू फाइनल है कि फिल्म देखने कोई भी परिवार ...,यह तो फाइनल है कि फिल्म देखने कोई भी परिवार के...,यह त फ इनल ह क फ लम दखन क ई भ पर व र क स थ नह ...,यह त फ इनल ह क फ लम दखन क ई भ पर व र क स थ नह ...
2,बिक पोस्क खुन्तेस्टेंट ने राजटरी मानवादिकार आय...,बिग्ग बॉस कंटेस्टेंट ने राष्ट्रीय मानव अधिकार ...,ब क प सक खनतसटट न र जटर म नव द क र आय ग म दरज ...,ब गग ब स कटसटट न र षटर य म नव अध क र म दरज क श...
3,प्रतीस परदा कितनी ही क्यो नहो कडी महनत करना कब...,प्रतिस्पर्धा कितनी ही क्यों न हो मेहनत करना कभ...,परत स परद क तन ह कय नह कड महनत करन कब न छ ड,परत सपरध क तन ह कय न ह महनत करन कभ न छ ड
4,"BOC, Aviation and Kingfisher Airlines के बीज क...",बीओसी एविएशन और किंगफिशर एयरलाइंस के बीच यह मा...,boc aviation and kingfisher airlines क ब ज क य...,ब ओस एव एशन और क गफ शर एयरल इस क ब च यह म मल ल...


In [ ]:
wer = jiwer.wer(list(data["reference_clean"]), list(data["hypothesis_clean"]))

print(f"WER: {wer * 100:.2f} %")

WER: 35.79 %
